In [1]:
import sys
from pathlib import Path

main_path = Path('..').resolve()
sys.path.append(str(main_path))

from src.dataset import MetaStockDataset
from src.utils import ARGProcessor
import numpy as np

In [2]:
setting_file = Path('.') / 'kdd.yml'

meta_args = ARGProcessor(setting_file=setting_file)
data_kwargs = meta_args.get_args(cls=MetaStockDataset)

In [3]:
meta_train = MetaStockDataset(meta_type='train', **data_kwargs)
# meta_test1 = MetaStockDataset(meta_type='test1', **data_kwargs)
# meta_test2 = MetaStockDataset(meta_type='test2', **data_kwargs)
# meta_test3 = MetaStockDataset(meta_type='test3', **data_kwargs)

Processing data and candidates for train: 100%|██████████| 40/40 [00:00<00:00, 42.27it/s]


In [4]:
tasks = meta_train.generate_tasks()

AMZN [ 205 1166 1560   97  625]
BA [1586 1691  352  862 1716]
BAC [682 374 220 803 376]
BHP [1346  350 1416  303 1716]
BRK-B [1433  674 1958  801 1875]
CHL [ 464 1376  431  195  758]
CMCSA [ 202 1292  804 1658 1829]
CVX [ 753  754 1345 1302  820]
DCM [1459  286  968  487  666]
DIS [1225  130  322  151  894]
DOW [  49 1855  783 1502 1163]
DUK [131 714 641 911 292]
GE [1782  512 1004 1894  700]
GOOGL [1299 1243 1225 1419  208]
HD [ 757 1869  132 1211 1910]
JNJ [ 555  237 1487  667   82]
JPM [ 280 1151 1070  663 1516]
KO [ 440  790   96 1977 1142]
MA [ 298 1659 1013  590  475]
MMM [1929   22 1223 1975  285]
MO [ 533   26  227 1053   63]
MRK [1336 1469  750 1101 1317]
MSFT [ 470  597  564  659 1967]
NGG [1429  383  976  634  410]
NTT [ 211  748 1520  731  981]
NVS [1949 1553  530  424  529]
PG [ 232  376 1278  563  884]
PTR [1624  666  824  982  623]
RIO [ 961  545 1554 1539 1082]
SO [1278  832 1789  688 1843]
SPY [1531 1129 1200  419 1740]
T [1928  967  354 1374  376]
TM [1019 1398 1468 1

C:\Users\simon\Desktop\Codes\SMILE\src\dataset.py:282: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  support = np.array(support)
C:\Users\simon\Desktop\Codes\SMILE\src\dataset.py:283: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  support_labels = np.array(support_labels)


In [7]:
meta_train.symbols[20]

'MO'

In [5]:
for s in tasks[5]['support']:
    print(s.shape)

(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5,)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)
(5, 4, 5, 11)


In [5]:
np.array(tasks[5]['support_labels']).shape

(40, 5, 4)

In [13]:
tasks[5].keys()

dict_keys(['support', 'support_labels', 'query', 'query_labels'])

In [14]:
for data in meta_train.iter_task(tasks[5]):
    break

In [16]:
data['query_labels']

array([[1],
       [1],
       [0],
       [0],
       [0]], dtype=uint8)

In [11]:
import json 
ps = list((meta_train.data_dir / 'kdd17/price_long_50').glob('*.csv'))
with (Path('../data').resolve() / 'kdd17/stock_universe.json').open('r') as file:
    universe_dict = json.load(file)

universe_key = 'known'
universe = universe_dict['0'][universe_key]
iterator = [p for p in ps if p.name.strip('.csv') in universe]

In [55]:
p = iterator[29]
stock_symbol = p.name.rstrip('.csv')
df_single = meta_train.load_single_stock(p)
df_single = df_single.loc[df_single["date"].between("2014-01-01", '2015-01-01')].reset_index(drop=True)

In [56]:
symbol = p.name.strip('.csv') # 'AMZN'
window_size = 5
n_support = 4
df_stock = meta_train.data[symbol]
labels_indices = meta_train.candidates[symbol]
labels_candidates = labels_indices[labels_indices >= window_size]
idx = meta_train.get_possible_idx(df_stock, labels_candidates)
labels_candidates = labels_candidates[idx:]

In [57]:
labels_candidates

array([  15,   16,   17, ..., 1982, 1983, 1984], dtype=int64)

In [60]:
df_stock.loc[:15, ['date', 'label']]

,date,label
0,2007-02-14,2
1,2007-02-15,0
2,2007-02-16,2
3,2007-02-20,2
4,2007-02-21,0
5,2007-02-22,2
6,2007-02-23,2
7,2007-02-26,2
8,2007-02-27,0
9,2007-02-28,1


In [61]:
y_q = np.array([labels_candidates[0]])
y_qs = y_q - window_size
query, query_labels = meta_train.generate_data(df_stock, y_start=y_qs, y_end=y_q)

In [62]:
y_qs

array([10], dtype=int64)

In [63]:
query.round(4)

array([[[-0.5848,  0.6405, -1.9772,  0.3073,  0.3073,  0.1392,  0.7574,
          0.9728,  1.0874,  0.943 ,  0.9697],
        [ 1.1299,  1.5819, -0.113 , -1.4202, -1.4202,  1.1243,  1.9407,
          2.2467,  2.4124,  2.2888,  2.3452],
        [ 0.8847,  1.0845, -0.1142, -1.017 , -1.017 ,  1.387 ,  2.5942,
          3.0403,  3.2848,  3.2297,  3.273 ],
        [-0.5072,  0.3099, -1.0989,  1.2842,  1.2842,  0.1071,  0.9862,
          1.5892,  1.8456,  1.8785,  1.8921],
        [ 0.    ,  0.6787, -0.2828, -0.3663, -0.3663,  0.2262,  1.0775,
          1.744 ,  2.0475,  2.1993,  2.1752]]])

In [64]:
df_stock.loc[10:14]

,date,open,high,low,close,adj_close,zd5,zd10,zd15,zd20,zd25,zd30,label
10,2007-03-01,-0.584793,0.640487,-1.977162,0.307265,0.307266,0.139237,0.757449,0.972802,1.087442,0.942961,0.969704,2
11,2007-03-02,1.129935,1.581912,-0.112997,-1.420212,-1.420211,1.124289,1.940672,2.246698,2.412423,2.288781,2.345161,0
12,2007-03-05,0.884695,1.084466,-0.114158,-1.016952,-1.016952,1.386984,2.594176,3.040332,3.284815,3.229709,3.273000,0
13,2007-03-06,-0.507188,0.309935,-1.098912,1.284249,1.284248,0.107070,0.986189,1.589176,1.845586,1.878530,1.892075,1
14,2007-03-07,0.000000,0.678725,-0.282814,-0.366303,-0.366304,0.226246,1.077489,1.743966,2.047510,2.199321,2.175243,2


In [26]:
df_stock.loc[:6, ['date', 'label']]

,date,label
0,2007-02-14,2
1,2007-02-15,0
2,2007-02-16,1
3,2007-02-20,0
4,2007-02-21,2
5,2007-02-22,2
6,2007-02-23,0


In [25]:
labels_candidates

array([   6,    7,    8, ..., 1982, 1983, 1984], dtype=int64)

In [38]:
def get_possible_idx(df_stock, labels_candidates):
    i = 0
    while i < len(labels_candidates):
        rise, fall = get_rise_fall(df_stock, labels_candidates, idx=i)
        if len(rise) + len(fall) == 4:
            break
        else:
            i += 1
    return i

def get_rise_fall(df_stock, labels_candidates, idx):
    df_check = df_stock.loc[labels_candidates[:idx], 'label'].sort_index(ascending=False)
    rise = df_check.index[df_check == meta_train.labels_dict['rise']][:(n_support // 2)].to_numpy()
    fall = df_check.index[df_check == meta_train.labels_dict['fall']][:(n_support // 2)].to_numpy()
    return rise, fall

In [39]:
# remove unpossible candidates
idx = get_possible_idx(df_stock, labels_candidates)
labels_candidates = labels_candidates[idx:]


In [41]:
y_q

array([[1277],
       [ 806],
       [ 407],
       [1164],
       [  66]], dtype=int64)

In [42]:
y_q = np.array(np.random.choice(labels_candidates, size=(5,), replace=False))
y_qs = y_q - window_size
query, query_labels = meta_train.generate_data(df_stock, y_start=y_qs, y_end=y_q)
support = []
support_labels = []
for q in y_q:
    q_idx = np.arange(len(labels_candidates))[labels_candidates == q][0]
    rise, fall = get_rise_fall(df_stock, labels_candidates, idx=q_idx)
    y_s = np.concatenate([fall, rise])
    y_ss = y_s - window_size
    data_s, label_s = meta_train.generate_data(df_stock, y_start=y_ss, y_end=y_s)
    data_s = np.array(data_s)
    support.append(data_s)
    support_labels.append(label_s)

In [45]:
for x in np.expand_dims(query_labels, 1):
    print(x)

[1]
[1]
[1]
[1]
[1]


In [46]:
x

array([1], dtype=uint8)

In [136]:
np.array(support).shape

(5, 4, 5, 11)

In [116]:
query_labels

[0, 1, 0, 1, 0]

In [120]:
q = y_q[0]
q_idx = np.arange(len(labels_candidates))[labels_candidates == q][0]
rise, fall = get_rise_fall(df_stock, labels_candidates, idx=q_idx)

In [124]:
y_s = np.concatenate([fall, rise])
y_ss = y_s - window_size

In [127]:
support, support_labels = meta_train.generate_data(df_stock, y_start=y_ss, y_end=y_s)


In [9]:
symbol = 'AMZN'
window_size = 5
n_shot = 2
df_stock = meta_train.data[symbol]
labels_indices = meta_train.candidates[symbol]
y_cand = labels_indices[labels_indices >= window_size]
n_rise = 0
n_fall = 0
support= []
support_sample = []
query = []
support_turn = True
query_turn = False
query_sample = []
for idx in y_cand:

    # ex. k = 2
    if support_turn and  n_rise < n_shot or n_fall < n_shot:
        if n_rise < 2 and df_stock['label'][idx] == 1:
            n_rise +=1
            support_sample.append(idx)
        elif n_fall < 2 and df_stock['label'][idx] == 0:
            n_fall +=1
            support_sample.append(idx)
        continue

    if n_rise == n_shot and n_fall == n_shot:
        support.append(support_sample)
        support_sample = []
        n_rise = 0
        n_fall = 0
        query_turn = True
        support_turn = False 

    if query_turn:
        query_sample.append(idx)
        query.append(query_sample)
        query_sample = []
        query_turn = False
        support_turn = True
        continue
support_idx_set = np.array(support)
query_idx_set = np.array(query)
print(len(support_idx_set), len(query_idx_set))

241 241


In [60]:
query_idx_set

array([[  14],
       [  20],
       [  28],
       [  52],
       [  57],
       [  63],
       [  71],
       [  77],
       [  83],
       [  89],
       [  95],
       [ 102],
       [ 112],
       [ 121],
       [ 128],
       [ 135],
       [ 144],
       [ 162],
       [ 168],
       [ 174],
       [ 181],
       [ 190],
       [ 196],
       [ 205],
       [ 211],
       [ 222],
       [ 239],
       [ 248],
       [ 253],
       [ 258],
       [ 268],
       [ 274],
       [ 279],
       [ 284],
       [ 291],
       [ 299],
       [ 304],
       [ 315],
       [ 321],
       [ 328],
       [ 336],
       [ 341],
       [ 350],
       [ 357],
       [ 364],
       [ 372],
       [ 377],
       [ 383],
       [ 389],
       [ 394],
       [ 401],
       [ 406],
       [ 411],
       [ 420],
       [ 427],
       [ 435],
       [ 443],
       [ 450],
       [ 455],
       [ 462],
       [ 467],
       [ 473],
       [ 479],
       [ 488],
       [ 493],
       [ 502],
       [ 5

In [6]:
labels_indices = self.candidates[symbol]
labels_candidates = labels_indices[labels_indices >= window_size]
y_s = np.array(sorted(np.random.choice(labels_candidates, size=(self.n_sample,), replace=False)))
y_ss = y_s-window_size
support, support_labels = self.generate_data(df_stock, y_start=y_ss, y_end=y_s)

# code for jumpped tags like [1(support), 0, 0, 1(query)]
# y_q = labels_indices[np.arange(len(labels_indices))[np.isin(labels_indices, y_s)] + self.n_lag]
y_q = y_s + self.n_lag
y_qs = y_s - window_size if self.keep_support_history else y_q - window_size
query, query_labels = self.generate_data(df_stock, y_start=y_qs, y_end=y_q)